In [1]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [7]:
def save_vid(file_directory, save_directory):
    new_width=224
    new_height=224

    # Read the input video file
    cap = cv2.VideoCapture(file_directory)

    # Create VideoWriter object for the output video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_path = os.path.join(save_directory)
    out = cv2.VideoWriter(output_path, fourcc, 20.0, (new_width, new_height))

    # Process each frame in the video
    while True:
        ret, frame = cap.read()

        if not ret:
            break

        # Resize the frame
        resized_frame = cv2.resize(frame, (new_width, new_height))

        # Write the resized frame to the output video
        out.write(resized_frame)

    # Release the video capture and writer objects
    cap.release()
    out.release()


In [15]:
import cv2
import os
import numpy as np

def resize_and_join_videos(input_paths, output_path, random_query, query_vid_path, summary_path, new_width, new_height):
    # Create VideoWriter object for the output video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, 20.0, (new_width, new_height))

    # Variables for capturing video duration across all clips
    start_time_global = None
    end_time_global = None

    for video_path in input_paths:
        cap = cv2.VideoCapture(video_path)
        
        # Variables for capturing video duration within each clip
        start_time_clip = None
        end_time_clip = None

        while True:
            ret, frame = cap.read()

            if not ret:
                break

            # Track the start time within the clip
            if start_time_clip is None:
                start_time_clip = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0

            # Track the start time globally
            if start_time_global is None:
                start_time_global = start_time_clip

            # Resize the frame
            resized_frame = cv2.resize(frame, (new_width, new_height))

            # Write the resized frame to the output video
            out.write(resized_frame)

            # Track the end time within the clip
            end_time_clip = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0

            # Track the end time globally
            end_time_global = end_time_clip

        # Release the video capture object
        cap.release()

        # Save the summary for the current clip
        save_vid(video_path, summary_path)

        # Calculate the duration in minutes within the clip
        duration_minutes_clip = (end_time_clip - start_time_clip) / 60.0 if start_time_clip is not None and end_time_clip is not None else None

    # Release the VideoWriter object
    out.release()

    # Calculate the duration in minutes globally
    duration_minutes_global = (end_time_global - start_time_global) / 60.0 if start_time_global is not None and end_time_global is not None else None

    return {'saved_directory': os.path.dirname(output_path),
            'query': random_query,  # Extract the crime type
            'moment': {'start': start_time_global, 'end': end_time_global},  # Include the moment interval
            'summary': summary_path}  # Include the summary file path

# Main code
path = r"D:\BARC\week_4\Anomaly Videos"
file_list = os.listdir(path)

result_dict = {}

summary_directory = r"D:\BARC\week_4\summary"

for j in range(2):
    list_of_path = []
    output_dir = r"D:\BARC\week_4\data_set"
    
    os.makedirs(output_dir, exist_ok=True)  # Create the output directory if it doesn't exist
    os.makedirs(summary_directory, exist_ok=True)  # Create the summary directory if it doesn't exist
    
    out_path = os.path.join(output_dir, f"output_{j}.mp4")  # Use j in the output file name
    summary_path = os.path.join(summary_directory, f"summary_{j}.mp4")  # Use j in the summary file name
    
    for i in file_list:
        # Construct the full video path
        full_video_path = os.path.join(path, i)
        
        random_file = np.random.choice(os.listdir(full_video_path))
        selected_path = os.path.join(full_video_path, random_file)
        
        list_of_path.append(selected_path)
    
    # Choose a random element from video_files
    random_idx = np.random.randint(0, len(list_of_path) - 1)
    random_query = file_list[random_idx]
    query_vid_path = list_of_path[random_idx]
    # Resize and join the videos
    result_info = resize_and_join_videos(list_of_path, out_path, random_query, query_vid_path, summary_path, new_width=224, new_height=224)

    # Update the result_dict with the information from resize_and_join_videos
    result_dict[j] = result_info

    print(f"Output saved to: {out_path}")
    print(result_info)


Output saved to: D:\BARC\week_4\data_set\output_0.mp4
{'saved_directory': 'D:\\BARC\\week_4\\data_set', 'query': 'Arson', 'moment': {'start': 0.0, 'end': 39.56666666666666}, 'summary': 'D:\\BARC\\week_4\\summary\\summary_0.mp4'}
Output saved to: D:\BARC\week_4\data_set\output_1.mp4
{'saved_directory': 'D:\\BARC\\week_4\\data_set', 'query': 'RoadAccidents', 'moment': {'start': 0.0, 'end': 210.0}, 'summary': 'D:\\BARC\\week_4\\summary\\summary_1.mp4'}
